In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\.virtualenvs\\course-management-platform-wiAsnpQu\\Lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-db2fb39d-8a7a-4f56-b1e5-70d4671ced1a.json']
Is test: False


In [2]:
from courses.models import *

In [3]:
from courses.scoring import update_faq_score, update_learning_in_public_score, update_leaderboard

In [4]:
from django.db.models import Prefetch

In [5]:
course = Course.objects.get(id=5)
course

<Course: Machine Learning Zoomcamp 2024>

In [6]:
homework = Homework.objects.get(course=course, slug='competition')
homework

<Homework: Machine Learning Zoomcamp 2024 - ML Zoomcamp 2024 Competition>

In [7]:
answers_prefetch = Prefetch("answer_set", queryset=Answer.objects.all())

In [8]:
submissions = Submission.objects.prefetch_related(answers_prefetch).filter(homework=homework)

In [20]:
for s in submissions:
    print(s.id) #, s.student)
    
    answers = list(s.answer_set.values("id", "question_id", "answer_text"))
    print(answers)

16254
[{'id': 92861, 'question_id': 198, 'answer_text': 'Sahil Sehgal'}, {'id': 92862, 'question_id': 199, 'answer_text': '41.0902'}, {'id': 92863, 'question_id': 200, 'answer_text': '36.7917'}]
None
15405
[{'id': 88052, 'question_id': 198, 'answer_text': 'Abdiaziz Mohamed'}, {'id': 88053, 'question_id': 199, 'answer_text': '27.5109'}, {'id': 88054, 'question_id': 200, 'answer_text': '21.2269'}]
None
13368
[{'id': 74381, 'question_id': 198, 'answer_text': 'Marko Golovko'}, {'id': 74382, 'question_id': 199, 'answer_text': '23.6956'}, {'id': 74383, 'question_id': 200, 'answer_text': ''}]
None
13370
[{'id': 74386, 'question_id': 199, 'answer_text': '18.7468'}, {'id': 74387, 'question_id': 200, 'answer_text': '16.6473'}, {'id': 74385, 'question_id': 198, 'answer_text': 'jyphotography'}]
None
14703
[{'id': 83457, 'question_id': 198, 'answer_text': 'dieego_gutierrez'}, {'id': 83458, 'question_id': 199, 'answer_text': '13.8908'}, {'id': 83459, 'question_id': 200, 'answer_text': ''}]
None
1541

## Custom logic

Updates

In [27]:
a = Answer.objects.get(id=62222)
a.answer_text = '0.9'
a.save()

Exclude

In [9]:
score_max = 8.9580
score_min = 24.4835

points_max = 20
points_min = 2

lower_better = score_max < score_min
lower_better

True

In [10]:
ignore = {11346}

In [11]:
def calculate_points(score):
    if lower_better:
        score_min_inv, score_max_inv = -score_min, -score_max
        score = -score
    else:
        score_min_inv, score_max_inv = score_min, score_max
    
    # print(f"Score: {score}, Max: {score_max_inv}, Min: {score_min_inv}")
    
    if score >= score_max_inv:
        return points_max
    elif score <= score_min_inv:
        return points_min
    
    score_ratio = (score - score_min_inv) / (score_max_inv - score_min_inv)
    # print(f"Score Ratio: {score_ratio}")
    points = points_min + score_ratio * (points_max - points_min)
    
    return round(points)


In [12]:
calculate_points(24.9580)

2

In [13]:
calculate_points((24.9580 + 8.9580) / 2)

11

In [14]:
calculate_points(8.9580)

20

In [15]:
for s in submissions:
    if s.id in ignore:
        continue

    print(s.id)
    
    answers_raw = list(s.answer_set.all())
    answers = {d.question_id: d for d in answers_raw}

    answer_name = answers[198]
    answer_name.correct = True
    answer_name.save()
    
    answer_score = answers[200]
    answer_score.correct = True
    answer_score.save()

    try:
        competition_score = float(answer_score.answer_text)
        points = calculate_points(competition_score)
    except ValueError as e:
        competition_score = 0
        points = 0

    s.questions_score = points

    lip_score = update_learning_in_public_score(s)
    faq_score = update_faq_score(s)
    
    s.total_score = points + lip_score + faq_score
    s.save()
    print(competition_score, points)

16254
36.7917 2
15405
21.2269 6
13368
0 0
13370
16.6473 11
14703
0 0
15416
18.6936 9
15371
9.5624 19
15422
24.4538 2
13581
0 0
15372
10.3003 18
15380
9.465 19
15401
21.2801 6
15413
21.3304 6
15989
12.0726 16
15267
19.2195 8
14949
9.2519 20
15933
0 0
15020
12.0726 16
15288
0 0
15307
18.6936 9
15493
21.2935 6
15341
0 0
15346
21.3007 6
15565
24.159 2


In [16]:
homework.state = HomeworkState.SCORED.value
homework.save()

In [18]:
update_leaderboard(course)

INFO Updating leaderboard for course 5
INFO Updated leaderboard in 1.09 seconds
